<a href="https://colab.research.google.com/github/rkeaveney/CPL_compiler/blob/main/id_adapter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code for downloading data from Waymo Open Dataset & extracting relevant data from .tfrecord files

In [ ]:
!git clone https://github.com/gdlg/simple-waymo-open-dataset-reader.git
!cd simple-waymo-open-dataset-reader/ && python setup.py install
!cp -r simple-waymo-open-dataset-reader/simple_waymo_open_dataset_reader/ .

In [ ]:
import os
import sys
import numpy as np
import math
import cv2
import matplotlib.pyplot as plt
import copy
import tensorflow as tf
from PIL import Image
import io
from simple_waymo_open_dataset_reader import utils as waymo_utils
from simple_waymo_open_dataset_reader import WaymoDataFileReader, dataset_pb2, label_pb2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from google.colab import auth
auth.authenticate_user()

Mounted at /content/drive


In [ ]:
!mkdir data
#!gsutil cp gs://waymo_open_dataset_v_1_4_1/individual_files/training/segment-10017090168044687777_6380_000_6400_000_with_camera_labels.tfrecord data
!gsutil cp gs://waymo_open_dataset_v_1_4_1/archived_files/training/training_0000.tar . && tar -xf training_0000.tar && rm training_0000.tar

mkdir: cannot create directory ‘data’: File exists
Copying gs://waymo_open_dataset_v_1_4_1/individual_files/training/segment-10017090168044687777_6380_000_6400_000_with_camera_labels.tfrecord...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

\ [1 files][  1.0 GiB/  1.0 GiB]                                                
Operation completed over 1 objects/1.0 GiB.                                      


In [ ]:
n_seg = 0
for tfrecord in tfrecords:
    datafile = WaymoDataFileReader(tfrecord)
    datafile_iter = iter(datafile)

    dir = "segment" + str(n_seg).zfill(4) + "/"
    
    n_fr = 0

    for frame in datafile_iter:
        filename = "s" + str(n_seg).zfill(4) + "f" + str(n_fr).zfill(4)

        # labels

        with open(dir + "labels/" + filename + ".txt", "w") as f:
            for obj in frame.camera_labels[0].labels:
                dtype = ""
                if obj.type == 2:
                    dtype = "0"
                elif obj.type == 4:
                    dtype = "1"
                elif obj.type == 1:
                    dtype = "2"
                elif obj.type == 3:
                    dtype = "3"
                cenX = obj.box.center_x / 1920
                cenY = (obj.box.center_y+320) / 1920
                width = obj.box.width / 1920
                length = obj.box.length / 1920

                left = "{:.6f}".format(cenX - length/2)
                top = "{:.6f}".format(cenY + width/2)
                right = "{:.6f}".format(cenX + length/2)
                bottom = "{:.6f}".format(cenY - width/2)

                f.write(dtype+"  "+left+"  "+top+"  "+right+"  "+bottom+"  "+obj.id+"\n")

        # yolo_labels

        with open(dir + "yolo_labels/" + filename + ".txt", "w") as f:
            for obj in frame.camera_labels[0].labels:
                dtype = ""
                if obj.type == 2:
                    dtype = "0"
                elif obj.type == 4:
                    dtype = "1"
                elif obj.type == 1:
                    dtype = "2"
                elif obj.type == 3:
                    dtype = "3"
                cenX = "{:.6f}".format(obj.box.center_x / 1920)
                cenY = "{:.6f}".format((obj.box.center_y+320) / 1920) 
                width = "{:.6f}".format(obj.box.width / 1920)
                length = "{:.6f}".format(obj.box.length / 1920)

                f.write(dtype+"  "+cenX+"  "+cenY+"  "+length+"  "+width + "\n")

        # images

        old_image = waymo_utils.decode_image(frame.images[0])
        old_image = Image.fromarray(old_image)
        new_image = Image.new("RGB", (1920, 1920))
        new_image.paste(old_image, (0, 320))
        new_image.save(dir + "images/" + filename + '.jpg')

        n_fr += 1
    print("Finished segment" + str(n_seg).zfill(4))
    #os.remove(tfrecord)
    n_seg += 1

Finished segment0000


In [ ]:
!for i in {98..99}; do cd /content/segment00$i && tar -czf segment00$i.tar.gz images labels yolo_labels; done
!for i in {100..197}; do cd /content/segment0$i && tar -czf segment0$i.tar.gz images labels yolo_labels; done